# Dissect phosim_cmpt, in an attempt to analyze the WFS data... 

In baseComcamLoop.py, we can prepare calibration products for WFS sensors with 

    fakeFlatDir = self._makeCalibsWfs(baseOutputDir)
    
which calls 

    makeGainImages.py --just_wfs  
    
We make ISR and postage stamp dirs as for ComCam (`self._makeDir(isrDir)` and `self._makeDir(isrDir)` are the same). 

We read the survey settings for `ts_phosim` from `ts_phosim/policy/surveySettings.yaml`  with `lsst.ts.phosim.Utility.getConfigDir`  - it just has 

    # Survey Settings for Catalogs

    # Filter Type
    filterType: ref

    # RA in degrees
    raInDeg: 0.0

    # Dec in degrees
    decInDeg: 0.0

    # Rotation Angle in degrees
    rotAngInDeg: 0.0


so that's for the entire LSSTCam (not jsut R22). 


Then we `_preparePhosimCmpt()` and `_prepareWepCalc()` .  I didn't see anything comcam-specific in  preparePhosim,  but in prepareWepCalc there is :

    wepCalc = WEPCalculationFactory.getCalculator(CamType.ComCam, isrDirPath)
    
So we'd need to set that to `CamType.LsstCam` (perhaps? list in https://github.com/lsst-ts/ts_wep/blob/master/python/lsst/ts/wep/Utility.py) 



Then in `_prepareOfcCalc()` I did the same : changed `ofcCalc = OFCCalculationFactory.getCalculator(InstName.COMCAM)` to `InstName.LSST`  (list from  `lsst.ts.ofc.Utility.InstName`  includes LSST, COMCAM, SH, CMOS )  .


Then we ingest the calibration products : 

    wepCalc.ingestCalibs(fakeFlatDir)
    
which is in https://github.com/lsst-ts/ts_wep/blob/master/python/lsst/ts/wep/ctrlIntf/WEPCalculation.py  (that doesn't seem to be ComCam-specific - it calls `ingestCalibs.py`  https://github.com/lsst-ts/ts_wep/blob/d32de1c93d067b48fb54dd0f4877d872e4b6f9e5/python/lsst/ts/wep/CamDataCollector.py#L29 


Then generating OPD has ComCam-specific  PhoSim argstring ` argString = phosimCmpt.getComCamOpdArgsAndFilesForPhoSim(cmdSettingFileName=opdCmdSettingsFile)` to which I prepend a list of sensors on which I want PhoSim to run. The argString is generated by https://github.com/lsst-ts/ts_phosim/blob/bea66c0714332038a46b0236fe418f898039004f/python/lsst/ts/phosim/PhosimCmpt.py#L356 


It seems that the only ComCam - specific part of that routine is https://github.com/lsst-ts/ts_phosim/blob/bea66c0714332038a46b0236fe418f898039004f/python/lsst/ts/phosim/PhosimCmpt.py#L387

    self.metr.setDefaultComcamGQ()
    
from https://github.com/lsst-ts/ts_phosim/blob/bea66c0714332038a46b0236fe418f898039004f/python/lsst/ts/phosim/OpdMetrology.py#L156 

so I would have to make a new part of `PhosimCmpt` that has 

    self.metr.setDefaultLsstGQ()
   
from https://github.com/lsst-ts/ts_phosim/blob/bea66c0714332038a46b0236fe418f898039004f/python/lsst/ts/phosim/OpdMetrology.py#L105

 - I call it `getLsstCamOpdArgsAndFilesForPhoSim`
 
Then the analysis of OPD data calls for 

    phosimCmpt.analyzeComCamOpdData(zkFileName=opdZkFileName,
                                            pssnFileName=opdPssnFileName)
                                            
from https://github.com/lsst-ts/ts_phosim/blob/bea66c0714332038a46b0236fe418f898039004f/python/lsst/ts/phosim/PhosimCmpt.py#L649   , and the LsstCam equivalent does not exist. The routine itself doesn't seem to be `ComCam` - specific, but as I dig in it turns out that `_writeOpdPssnFile` calls  `pssnList, gqEffPssn = self._calcComCamOpdPssn()` and ` effFwhmList, gqEffFwhm = self._calcComCamOpdEffFwhm(pssnList)` - both of which are ComCam - specific. 

So I make  



I make it and call it `analyzeLsstCamOpdData`.


